In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import warnings
import pandas as pd
import numpy as np
from tifffile import imread
from nd2reader import ND2Reader
from scipy.ndimage import zoom

import napari
from napari import Viewer
from napari import notification_manager 
from magicgui.widgets import Container,Label,Dropdown
from magicgui import magicgui

sys.path.append('../libraries')
import general_functions as gen
import napari_build_viewer as bv_f
import input_functions as inp_f

In [ ]:
warnings.filterwarnings('ignore')

In [2]:
info_file_path = r'Z:\COOK_LAB\MJ\Movie_10\10\code\scripts\20231118_10_rawinfo.txt'

## Define experiment

In [3]:
# read the file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

# read in the data frame
df = pd.read_pickle(os.path.join(exp_dir,'df',df_name))

# get info about the channels
channel_list = inp_f.read_channels(info_lines,check=True,df=df,exp_dir=exp_dir)

# get info about the tags (for annotating tracks)
tag_list = inp_f.read_tags(info_lines,df=df)

# get info about the tags (for annotating tracks)
flag_list = inp_f.read_flags(info_lines,df=df)

# get info about what will be plotted
graph_list = inp_f.read_graphs(info_lines,df,channel_list)

# get info about the general settings
time_threshold,small_im_size,label_contour,gen_track_columns = inp_f.read_settings(info_lines)

# get info about the caluclations 
object_properties = inp_f.read_properties(info_lines)
# consider pushing properties_ring too

## Extract info from the data frame

In [4]:
# sort 
df = df.sort_values(by=['track_id','t'])

In [ ]:
'''
# Performs downscaling on the dataframe. This is done in two parts, the mathematical reduction of distance based units
# and the downsampling of the 'image' boolean array by pooling. 
# Two variables must be defined, one for pooling and the other for mathematical division. They must align.
#i.e. if you are reducing by a factor of 2, your pooling factor must be 0.5 . If reducing by a factor of 4, pooling factor must be 0.25

#Downscale the 'image' column boolean array by this number
boolean_factor = 1

#Divide cols_to_reduce by this number. Set to 1 if you do not want to perform scaling
scaling_factor = 1

#Define which columns in the dataframe must be divided by the downscale_factor
cols_to_scale = ['area', 'centroid-0', 'centroid-1', 'major_axis_length', 'minor_axis_length', 'bbox-0', 'bbox-1', 'bbox-2', 'bbox-3',
                  'centroid-0_ring', 'centroid-1_ring', 'size_x', 'size_y', 'x', 'y']


# Check if 'downscaled' column exists
if 'downscaled' not in df.columns:
    df['downscaled'] = 0

# Check downscaled coloumn for consistency
unique_values = df['downscaled'].unique()

if scaling_factor != 1:

    # downscaled coloumn passes consistency check,, proceed to actual downscale check
    if len(unique_values) == 1:
        val = unique_values[0]
        
        # Check the value in the 'downscaled' column
        if val == 0:
            # Function to downsample a boolean image by a given factor
            def downsample_image(image, factor=boolean_factor):
                """Downsample a boolean image by a given factor."""
                return zoom(image, factor, order=1) > 0
            
            df[cols_to_scale] = df[cols_to_scale] / scaling_factor
            
            # Downscale the images in the 'image' column
            df['image'] = df['image'].apply(downsample_image)
            
            # Set the 'downscaled' column to 1, indicating that the dataframe has been downscaled
            df['downscaled'] = 1
            print(f'Dataframe downscaled by a factor of {downscale_factor}')
        elif val == 1:
            print("The dataframe has already been downscaled. This operation should not be run multiple times. Downscaling not performed")
        elif val == 2:
            print("The dataframe has already been downscaled and upscaled. This operation should not be run multiple times. Downscaling not peformed")
    else:
        print("The 'downscaled' column has multiple unique values. Please ensure its consistency.")
else:
    print('Scaling_factor set to 1, no scaling requested. Scaling not performed')

Expected execution time < 1min.

In [5]:
%%time

# generate labels layer
#This is currently using a modified version of labels_from_df that facilitates scale reduction by padding the image column with zeros to ensure alignment
labels = gen.labels_from_df(df)

CPU times: total: 5.02 s
Wall time: 5.02 s


In [6]:
# generate data for the tracking layer
data,properties,graph = gen.trackData_from_df(df,col_list = gen_track_columns)

In [7]:
# create data for tagging
tag_data = gen.tags_from_df(df,tag_list)

In [8]:
graph

{}

In [9]:
df.to_pickle(os.path.join(exp_dir,'df',df_name))

## Read in images

Expected execution time ~ 10s/1GB

In [10]:
%%time

for ind,ch in enumerate(channel_list):
    
    im_path = os.path.join(exp_dir,'data',ch['file_name'])
    c = ch['channel_in_file']
    
    channel_list[ind]['image'] = inp_f.open_movie(im_path,c)

CPU times: total: 4.08 s
Wall time: 10min 7s


## Create a viewer

In [11]:
viewer = napari.Viewer()

# add a helper layer
layer_mod = viewer.add_points([],name='Helper Points',face_color='red',ndim=3)

# add tracks annotations
for tag,tag_points in zip(tag_list,tag_data): 
    
    viewer.add_points(tag_points,name=tag['tag_name'],face_color=tag['tag_color'],opacity = 1,ndim = 3)

# add image layers

for ind,ch in reversed(list(enumerate(channel_list))):
    
    viewer.add_image(ch['image'],name=ch['channel_name'],colormap=ch['color'],blending='additive')

# add a tracking layer
track_layer=viewer.add_tracks(data, properties=properties,graph={},name='Tracking',color_by='track_id')
track_layer.display_id=True
    
# add a labels layer
labels_layer = viewer.add_labels(labels,name='Labels',opacity = 0.4)

In [12]:
# create a plot widget
t_max = viewer.dims.range[0][1]
plot_widget = bv_f.build_lineage_widget(t_max)

# create a list of promising tracks
promise_list = [int(x) for x in set(df.loc[df.promise==True,'track_id'])]
promise_list.append(0)
promise_list.sort()

## Build viewer functionality

In [13]:
# inject global variables to the module

global_variables = ['viewer','plot_widget',
                    'exp_dir','df_name','df',
                    'channel_list','graph_list',
                    'object_properties','promise_list',
                    'time_threshold','flag_list',
                    'tag_list','gen_track_columns',
                    'small_im_size',
                    'label_contour'
                   ]

for var in global_variables:
    
    exec(f'bv_f.{var} = {var}')

In [14]:
######################################################################
# add saving button
save_data = magicgui(bv_f.save_data, call_button='Save Data')
viewer.window.add_dock_widget(save_data,area='right')

######################################################################
# add right-click to make a label active
labels_layer.mouse_drag_callbacks.append(bv_f.select_label)

######################################################################
# add label modifications

mod_label = magicgui(bv_f.mod_label,call_button='Modify Label')
viewer.window.add_dock_widget(mod_label,area='right')

mod_key = viewer.bind_key('Shift-Enter',overwrite=True,func=bv_f.mod_label)

######################################################################
# add track modifying buttons

cut_track = magicgui(bv_f.cut_track, call_button='Cut Track')
merge_track = magicgui(bv_f.merge_track, call_button='Merge Track')
connect_track = magicgui(bv_f.connect_track, call_button='Connect Track')

container_tracks = Container(widgets=[cut_track,merge_track,connect_track],labels=False)
viewer.window.add_dock_widget(container_tracks,area='right') 

######################################################################
# add right-click toggle track tags
for tag_name in [x['tag_name'] for x in tag_list]:
    
    viewer.layers[tag_name].mouse_drag_callbacks.append(bv_f.toggle_track)

######################################################################
# add navigation in the track
begin_button= magicgui(bv_f.go_to_track_beginning,call_button="<")
end_button= magicgui(bv_f.go_to_track_end,call_button=">")
center_button = magicgui(bv_f.center_the_cell,call_button="<>")

c = Container(widgets=[begin_button,center_button, end_button],layout='horizontal',labels=False)
viewer.window.add_dock_widget(c,area='right',name='Navigate Track')
    
######################################################################
# add small stack display button

stack_button = magicgui(bv_f.show_stack, call_button='Show Stack')
viewer.window.add_dock_widget(stack_button,area='right')

#####################################################################
# button for next available track
next_label_button = magicgui(bv_f.next_label, call_button='Next Label')
viewer.window.add_dock_widget(next_label_button,area='left',name='Next Available Label')

######################################################################
# add navigation over promising tracks

widget_label_promise = Label(value=f'Number of promising tracks: {(len(promise_list)-1)}', label="Promising tracks:")
widget_list_promise = Dropdown(choices=promise_list, value = 0)
bv_f.select_promising_track = widget_list_promise.changed.connect(bv_f.select_promising_track)

prom_c = Container(widgets=[widget_label_promise,widget_list_promise],layout='horizontal',labels=False)

viewer.window.add_dock_widget(prom_c,area='left',name='Choose a promising track')

########################################################################
# add lineage graph
viewer.window.add_dock_widget(plot_widget,area='bottom',name='family')

# connect lineage graph update
labels_layer.events.selected_label.connect(bv_f.update_lineage_display)

# init family line
bv_f.init_family_line(0)


########################################################################
# share menu specific elements
widget_variables=['promise_list','widget_label_promise','widget_list_promise']
for var in widget_variables:
    
    exec(f'bv_f.{var} = {var}')

C:\Users\gases\Desktop\tracking_pipeline-master\notebooks\general_functions.py:45: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  x = int(cellData['centroid-0'])
C:\Users\gases\Desktop\tracking_pipeline-master\notebooks\general_functions.py:46: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  y = int(cellData['centroid-1'])


<font size = '6'>Upscale final tracked dataframe for downstream 4i alignment </font>

In [ ]:
#Add in a bit to check that promising tracks have all been cleared.If you attempt to upscale with promising tracks left, you'll
#need to set a variable equal to the number of promising tracks remaining to override the check and proceed with upscaling.

promising_track_override = 0

# Check if 'downscaled' column exists
if 'downscaled' not in df.columns:
    df['downscaled'] = 0

# Check downscaled coloumn for consistency
unique_values = df['downscaled'].unique()
# Determine if all promising tracks have been tracked
num_promising_tracks = df['promise'].sum()

if scaling_factor != 1:
    
    if len(unique_values) == 1:
        val = unique_values[0]
        
        if val == 0:
            print("The dataframe has not been downscaled. This operation is not required. Upscaling not performed")
        elif val == 1:
            # Check if there are any promising tracks or if the override is set
            if num_promising_tracks == 0 or num_promising_tracks == promising_track_override:
                # Function to upsample a boolean image by a given factor
                def upscale_image(image, factor=scaling_factor):
                    return zoom(image, factor, order=0) > 0.5
                
                df[cols_to_scale] = df[cols_to_scale] * scaling_factor
                
                # Upscale the images in the 'image' column
                df['image'] = df['image'].apply(upscale_image)
                
                # Set the 'downscaled' column to 1, indicating that the dataframe has been upscaled
                df['downscaled'] = 2  # Adjusted the value to 2 to indicate upscaling
                print(f'Dataframe upscaled by a factor of {scaling_factor}')
                #Save upscaled dataframe
                df.to_pickle(os.path.join(exp_dir,'df',df_name))
                print('The upscaled dataframe has been saved')
            else:
                print(f"The dataframe has promising tracks remaining. Cannot proceed with upscaling. Set the variable promising_track_override to {num_promising_tracks} to override.")
        elif val == 2:
            print("The dataframe has already been downscaled and upscaled. This operation should not be run multiple times. Upscaling not performed.")
    else:
        print("The 'downscaled' column has multiple unique values. Please ensure its consistency.")
else:
    print('Scaling_factor set to 1, no scaling requested. Scaling not performed')
    #Save final dataframe
    df.to_pickle(os.path.join(exp_dir,'df',df_name))
    print('The dataframe has been saved')